# ETL 2 - Creates Features, Cleans Data

This ETL module combines Zipcode informtion, including the cluster label that was produced in Model 4, with weather data and 311 call data.

In [1]:
import pandas as pd
import numpy as np
print('Libraries imported.')


Libraries imported.


## Zipcodes

The zipcode data includes the cluster data that was generated in Model 4

### Feature Creation

The new feature was the cluster data generated in Model 4


In [2]:
df_zips = pd.read_csv("csvs/labeled_zips.csv", index_col = 0, usecols=[ 1,12], dtype={0:'str', 1:'str', 12:'int'})

df_zips.head()

,kmeans_label
Zip,
10001,2
10002,0
10003,0
10004,2
10005,2


## Top Ten data

This has the top ten complaints registered over the past five years, includes zipcode and date.

In [3]:
df_311 = pd.read_csv("csvs/top_ten_w_zips.csv", index_col=0, usecols=[0,3,5,6], dtype={0:'int64', 3:'str', 5:'int64', 6:'str'})#, parse_dates=['Date Only'])

df_311.head()


/Users/dtt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Complaint Type,Incident Zip,Date Only
Unique Key,,,
31723508,Noise - Residential,10023,2015-10-10
31724314,Noise - Residential,10023,2015-10-10
31724946,HEAT/HOT WATER,10023,2015-10-10
31725751,HEAT/HOT WATER,10023,2015-10-10
31726082,Noise - Residential,10023,2015-10-10


## Weather Data

Previous analysis showed that all we really need for this analysis is the average daily temperature

In [4]:
df_weather = pd.read_csv("csvs/daily_weather_nyc.csv",  usecols=[0,1,4], dtype={0:'str', 1:'str', 2:'int'})
df_weather["Date Only"] = df_weather["DATE"].apply(lambda x: x[:10])
df_weather.drop(columns=['DATE','Unnamed: 0'], inplace=True)
df_weather.set_index("Date Only", inplace=True )
df_weather.head()

,DailyAverageDryBulbTemperature
Date Only,
2015-01-01,33
2015-01-02,39
2015-01-03,38
2015-01-04,49
2015-01-05,35


## Join Data

So now we're going do an inner join on the 311 data with the zipcode data, and then with the weather data. This provides us with a smaller data set. The zipcode data only contains those zipcodes in NYC, so the join will remove a lot of 311 calls that have unwanted or invalid zipcodes.

We finish by contatentating date with temperature, and also concatenating zip code with the cluster label. This will preserve these data after when we do the pivot table.

In [5]:
#
df_j = df_311.join(df_zips, on="Incident Zip", how="inner")
df_j = df_j.join(df_weather, on="Date Only", how="inner")
#df_j['WE'] = (pd.to_datetime(df_311['Date Only']).dt.dayofweek).isin([5,6])
#df_j['DOW'] = (pd.to_datetime(df_311['Date Only']).dt.dayofweek)
#df_j['Complaint ID'] = df_j['Complaint Type']
#df_j['Complaint ID'] = df_j['Complaint ID'].map({'Blocked Driveway': 1,'HEAT/HOT WATER': 2,'Illegal Parking': 3,'Noise - Residential': 4,'Noise - Street/Sidewalk': 5,'Street Condition': 6,'Street Light Condition': 7,'UNSANITARY CONDITION': 8,'Water System': 9,'Request Large Bulky Item Collection': 10})
df_j['Date_Temp'] = df_j['Date Only'].astype(str)+'_'+df_j['DailyAverageDryBulbTemperature'].astype(str)
#df_j['Zip_Group'] = df_j['Incident Zip'].astype(str)+'_'+df_j['kmeans_label'].astype(str)




## Pivot Table

Create a pivot table showing the number of each type of call for each date and each zipcode

In [18]:
for i in range(0,6):
    df_b = pd.DataFrame(df_j)[df_j['kmeans_label']==i]
    df_b.drop(columns=['kmeans_label','Date Only','Incident Zip','DailyAverageDryBulbTemperature'], inplace=True)
    df_b['X'] = 1
    pt = pd.pivot_table(df_b,  index=['Date_Temp'], columns=['Complaint Type'],
                          aggfunc=np.sum, fill_value=0)
    pt.to_csv("csvs/group{}_temp_311.csv".format(i))

In [19]:
pt

X                                      \
Complaint Type Blocked Driveway Illegal Parking Noise - Residential   
Date_Temp                                                             
2015-01-13_27                 0               0                   0   
2015-04-13_59                 0               0                   0   
2015-06-09_76                 0               1                   0   
2015-06-11_81                 0               0                   0   
2015-07-31_81                 0               0                   0   
2015-11-17_46                 0               0                   0   
2016-01-01_38                 0               0                   0   
2016-02-19_32                 0               1                   0   
2016-05-13_61                 0               0                   0   
2016-06-06_74                 0               0                   0   
2016-07-21_81                 0               0                   0   
2016-07-22_84                 0               0                   0   
2016-07-27_83                 0               0                   0   
2016-08-01_75                 0               0                   0   
2016-08-08_79                 0               0                   0   
2016-08-24_78                 0               0                   0   
2016-08-26_83                 0               0                   0   
2016-09-28_63                 0               0                   0   
2016-10-12_60                 0               0                   0   
2016-10-13_60                 0               0                   0   
2016-10-18_74                 0               0                   0   
2016-10-21_63                 0               0                   0   
2016-10-26_45                 0               0                   0   
2016-10-29_52                 0               0                   0   
2016-11-18_54                 0               1                   0   
2017-01-10_34                 0               0                   0   
2017-02-25_54                 0               0                   0   
2017-03-04_24                 0               1                   0   
2017-03-07_49                 0               0                   0   
2017-03-24_47                 0               0                   0   
...                         ...             ...                 ...   
2018-07-12_74                 0               0                   0   
2018-07-13_76                 0               0                   0   
2018-07-18_76                 0               0                   0   
2018-07-22_75                 0               0                   0   
2018-09-04_85                 0               1                   0   
2018-10-06_66                 0               1                   0   
2018-10-08_69                 1               0                   0   
2018-10-09_72                 0               0                   0   
2018-11-12_43                 0               0                   0   
2018-11-29_42                 1               0                   0   
2018-12-05_33                 0               0                   0   
2018-12-09_32                 0               0                   0   
2018-12-26_38                 0               0                   0   
2018-12-30_37                 1               0                   0   
2019-02-04_51                 0               0                   0   
2019-02-08_44                 0               0                   0   
2019-02-09_28                 0               0                   0   
2019-02-11_34                 0               0                   0   
2019-02-15_51                 0               0                   0   
2019-03-03_38                 0               0                   0   
2019-03-15_62                 0               0                   0   
2019-03-21_44                 0               0                   0   
2019-04-01_40                 0               0       

## Pivot Table Save

Save the Pivot Table as we are done

In [ ]:
pt.to_csv("csvs/group_temp_311.csv")